In [ ]:
# Step 1: Copy submission script to default location
# Clone the anemoi-house repository into ./src before running for security:
# git clone https://github.com/NOAA-PSL/anemoi-house ./src
# Cloning ensures only vetted code is executed.
!bash ./copy_submit_training.sh

Copied submit_training_aml.sh to ./src/anemoi-house/replay/atmosphere-subsampled/p0/training


Clone the [anemoi-house](https://github.com/NOAA-PSL/anemoi-house) repository into the `./src` folder before running the script. This ensures the code is retrieved from a trusted source for security.

### Step 2: Setup local configuration

Configure workspace and the resource group

Login if necessary:

In [ ]:
#!az login

This command uses the Azure CLI to authenticate with Azure. Because it starts with `#!`, it is commented out to avoid automatic execution when running the notebook. Remove the `#` if you want to log in from within the notebook. Authentication is required before any workspace operations can succeed.

Set your default Workspace and Resource Group:

In [ ]:
!az configure --defaults workspace=<workspace name> group=<resource group name>

The command sets your default Azure ML workspace and resource group. After running it, subsequent CLI commands will implicitly target these defaults, so you do not need to pass the workspace or group every time. Replace the placeholders with the actual names used in your Azure subscription.

### Step 3: Create AML environment to capture Anemoi python dependencies

#### Generate environment configuration yaml

In [ ]:
%%writefile anemoihouse_env.yaml
$schema: https://azuremlschemas.azureedge.net/latest/environment.schema.json
name: anemoihouse
build:
  path: environment

This cell writes the YAML configuration for the custom Azure ML environment. The file references the `environment` directory containing the Dockerfile and dependency list. Creating this file is the first step toward registering a reproducible environment for training.

#### Create the environment using CLI

In [ ]:
!az ml environment create --file anemoihouse_env.yaml

With the environment YAML on disk, this command registers the environment in Azure ML. It ensures that the Docker image and conda dependencies described earlier are available for subsequent jobs.

### Step 4: Start the training job

In [ ]:
%%writefile anemoi_training_job.yaml
$schema: https://azuremlschemas.azureedge.net/latest/commandJob.schema.json
code: ./replay/atmosphere-subsampled/p0/training
command: >-
  echo ${{inputs.data}} ${{outputs.output_dir}}; ./submit_training_aml.sh ${{inputs.data}} ${{outputs.output_dir}};
environment: azureml:anemoihouse@latest 
resources:
  instance_count: <number of VMs in a cluster(e.g.: 1,2,3,...)>
inputs:
  data: 
    type: uri_folder
    mode: ro_mount
    path: azureml:<AzureML data asset name>:<data asset version>
outputs:
  output_dir: 
    type: uri_folder
    mode: upload

distribution:
  type: mpi
  process_count_per_instance: <number of GPUs/VM>
services:
    my_vs_code:
      type: vs_code
    my_jupyter_lab:
      type: jupyter_lab
compute: azureml:<cluster name>
display_name: anemoitraining
experiment_name: anemoi
description: anemoi training job


Here we create the YAML specification for the training job. It defines input data, output paths, the environment to use, compute resources, and services like VS Code. Fill in the placeholders to match your dataset, cluster, and GPU configuration before submitting the job.

In [ ]:
!az ml job create -f anemoi_training_job.yaml

This final command submits the job definition to Azure ML. Azure ML will schedule the run on the specified compute cluster using the environment and parameters described in the previous cells.